# Author's Comment
The Comments below is wrtten by the kaggle user, who uploaded this dataset.
First thing I want to tell you is that the purpose of this notebook is to predict one's victory based on the conditions written on the datasets. And it'll be achieved by a classfication, and trendy XGboost model to get it better.

The second thing is that, I repeat these on my notebook just to let you know that I just followed the guidlines referred by the user.
If I continue to revise my model and work on it, it'd be much better to predict the victory of the battle.




## History of Battle
This dataset is a cleaned-up and modernized version of "CAA Database of Battles, Version 1990", shortnamed "CDB90". It contains information on over 600 battles that were fought between 1600 AD and 1973 AD. Descriptive data include battle name, date, and location; the strengths and losses on each side; identification of the victor; temporal duration of the battle; and selected environmental and tactical environment descriptors (such as type of fortifications, type of tactical scheme, weather conditions, width of front, etc.).


## Content
The data contained therein is split across several files. The most important of these is battles.csv, which is lists and gives information about the battles themselves. Files marked enum describe the keys used by specific fields. Other files provide additional context.

## Inspiration
-How often were battles fought in various weather conditions?

-How often did an attacker or defender achieve the element of surprise?

-Did it have a significant effect on the outcome?

-Did prepared fortifications have a significant effect on outcomes?


# Features

Before we get to the project, I'll make a referrence of the features to make it easier to understand the table.

-aeroa: air superitority

-cea: how much the actor favors attack or defense

-codead: total strength

-crit: ???

-post: how much defense prepared

-postype: posture taken

-pri: the form of tactics

-wina: 0-draw/1-attacker win/ -1 - attacker loss

-reso: result of the battle

-surpa: how much surprise achieved by attacker/defender

front_width: the features in it are unknown


### Terrain
-terra1 : None/Rugged/Rollig/Flat

-terra2 : Bare/Desert/Heavily Wooded/Mixed

-terra3 : None/ Marsh or Swamp/ Urban /Dunes

### Weather


-wx1: Dry/wet

-wx2: Heavy Precipitatiion/Sunny/Light Precipitation/Overcast

-wx3: hot/cold/temperate

-wx4: Spring/Summer/Fall/Winter

-wx5: Tropical/Desert/Temperate



# Data Overview

As mentioned above, the first phase will be just using 4 features.

-weather

-terrain

-the element of surprise

-fortification



Any other background knowledge will be used later on.
Before that, let us first overview the dataset.

# Importing Libraries

In [1249]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


#data processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import xgboost as xgb
import plotly.graph_objects as go

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/database-of-battles/enum_post.csv
/kaggle/input/database-of-battles/battle_actors.csv
/kaggle/input/database-of-battles/battles.csv
/kaggle/input/database-of-battles/enum_aeroa.csv
/kaggle/input/database-of-battles/enum_pri.csv
/kaggle/input/database-of-battles/terrain.csv
/kaggle/input/database-of-battles/belligerents.csv
/kaggle/input/database-of-battles/enum_terra2.csv
/kaggle/input/database-of-battles/weather.csv
/kaggle/input/database-of-battles/enum_surpa.csv
/kaggle/input/database-of-battles/enum_reso.csv
/kaggle/input/database-of-battles/version.csv
/kaggle/input/database-of-battles/enum_wx4.csv
/kaggle/input/database-of-battles/enum_wx3.csv
/kaggle/input/database-of-battles/enum_terra1.csv
/kaggle/input/database-of-battles/front_widths.csv
/kaggle/input/database-of-battles/commanders.csv
/kaggle/input/database-of-battles/enum_cea.csv
/kaggle/input/database-of-battles/enum_wx5.csv
/kaggle/input/database-of-battles/enum_wx2.csv
/kaggle/input/database-of-battles/bat

In [1250]:
#algorithms
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, accuracy_score

#dataframe display settings
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)

# Exploring Dataset

## Tables

In [1251]:
battle = pd.read_csv('/kaggle/input/database-of-battles/battles.csv')
display(battle.tail())
print(battle.columns)

,isqno,war,name,locn,campgn,postype,post1,post2,front,depth,time,aeroa,surpa,cea,leada,trnga,morala,logsa,momnta,intela,techa,inita,wina,kmda,crit,quala,resa,mobila,aira,fprepa,wxa,terra,leadaa,plana,surpaa,mana,logsaa,fortsa,deepa,is_hero,war2,war3,war4,war4_theater,dbpedia,cow_warno,cow_warname,war_initiator,parent
655,656,"ARAB-ISRAELI, 1948",JERUSALEM CORRIDOR,WEST BANK,JERUSALEM CORRIDOR,9,PD,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,"ARAB-ISRAELI, 1948","ARAB-ISRAELI, 1948",Arab-Israeli War of 1948-1949,NaN,http://dbpedia.org/resource/Operation_Danny,148.0,Arab-Israeli War of 1948-1949,0,NaN
656,657,WORLD WAR II,ARDENNES,NORTHWEST EUROPE,ARDENNES 1940,9,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,WORLD WAR II (FRANCE 1940),WORLD WAR II,World War II of 1939-1945,Western Front,NaN,139.0,World War II of 1939-1945,1,NaN
657,658,WORLD WAR II,CAMBRAI,NORTHWEST EUROPE,CAMBRAI,9,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,48.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,WORLD WAR II (FRANCE 1940),WORLD WAR II,World War II of 1939-1945,Western Front,NaN,139.0,World War II of 1939-1945,1,NaN
658,659,WORLD WAR II,ARRAS,NORTHWEST EUROPE,ARRAS,9,HD,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,WORLD WAR II (FRANCE 1940),WORLD WAR II,World War II of 1939-1945,Western Front,NaN,139.0,World War II of 1939-1945,0,NaN
659,660,WORLD WAR II,BOOS,NORTHWEST EUROPE,BOOS,9,PD,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,WORLD WAR II (FRANCE 1940),WORLD WAR II,World War II of 1939-1945,Western Front,NaN,139.0,World War II of 1939-1945,0,NaN


Index(['isqno', 'war', 'name', 'locn', 'campgn', 'postype', 'post1', 'post2',
       'front', 'depth', 'time', 'aeroa', 'surpa', 'cea', 'leada', 'trnga',
       'morala', 'logsa', 'momnta', 'intela', 'techa', 'inita', 'wina', 'kmda',
       'crit', 'quala', 'resa', 'mobila', 'aira', 'fprepa', 'wxa', 'terra',
       'leadaa', 'plana', 'surpaa', 'mana', 'logsaa', 'fortsa', 'deepa',
       'is_hero', 'war2', 'war3', 'war4', 'war4_theater', 'dbpedia',
       'cow_warno', 'cow_warname', 'war_initiator', 'parent'],
      dtype='object')


In [1252]:
terrain = pd.read_csv('/kaggle/input/database-of-battles/terrain.csv')
display(terrain.tail())
print(terrain.columns)

,isqno,terrano,terra1,terra2,terra3
784,656,1,R,B,NaN
785,657,1,R,M,NaN
786,658,1,R,M,NaN
787,659,1,R,M,NaN
788,660,1,R,M,NaN


Index(['isqno', 'terrano', 'terra1', 'terra2', 'terra3'], dtype='object')


In [1253]:
weather = pd.read_csv('/kaggle/input/database-of-battles/weather.csv')
display(weather.tail())
print(weather.columns)

,isqno,wxno,wx1,wx2,wx3,wx4,wx5
782,656,1,D,NaN,H,S,D
783,657,1,W,L,T,$,T
784,658,1,W,L,T,$,T
785,659,1,D,NaN,T,$,T
786,660,1,W,L,T,S,T


Index(['isqno', 'wxno', 'wx1', 'wx2', 'wx3', 'wx4', 'wx5'], dtype='object')


There are only 4 features that we're interested in, so we are gonna merge these tables and have only the features left.

-weather
-terrain
-fortification
-element of surprise achieved or not

In [1254]:
#The revised dataset that contains all information needed

df = pd.merge(battle, terrain , on="isqno")
df = pd.merge(df, weather, on = "isqno")
df.set_index('isqno', inplace = True)
df = df[['surpa', 'post1', 'post2', 'wx1', 'wx2', 'wx3', 'wx4', 'wx5', 'terra1', 'terra2', 'terra3', 'wina']]

display(df.tail())

KeyboardInterrupt: 

In [ ]:
# Based on what I searched for, Nan datas in 'wina' are supposed to be -1.(attacker loss)
df['wina'] = df['wina'].fillna(-1)
display(df)

The reason the amount of the datas increased is that each war consists of a few battles, and each battle has its own condition.
But it seems that the datas sharing same isqno have the same 'wina' value, so we don't need to set up the function to fix it.

* Different condtions but same result! Need to think about it one day

## Missing Data


Now we'll check missing data and se the tables in the form easy to analyse.

### Element of Surprise achieved or not

In [ ]:
import missingno as msno
#checking if there's any empty values on the chart.


#checking if there's any empty values on the chart.
surprise = df[['surpa', 'wina']]

msno.matrix(surprise)
plt.show()

print(surprise.isnull().sum())

### Terrain

In [ ]:
import missingno as msno
#checking if there's any empty values on the chart.


#checking if there's any empty values on the chart.
terrain = df[['terra1', 'terra2', 'terra3', 'wina']]

msno.matrix(terrain)
plt.show()

print(terrain.isnull().sum()) 

### Weather

In [ ]:
import missingno as msno
#checking if there's any empty values on the chart.


#checking if there's any empty values on the chart.
weather = df[['wx1', 'wx2', 'wx3', 'wx4', 'wx5','wina']]

msno.matrix(weather)
plt.show()

print(weather.isnull().sum())

### Fortification

In [ ]:
import missingno as msno
#checking if there's any empty values on the chart.


#checking if there's any empty values on the chart.
fortification = df[['post1', 'post2' ,'wina']]
msno.matrix(fortification)
plt.show()

print(fortification.isnull().sum())

## Data Overview

### Wina Reference

    * -1 : attacker loss
    * 0 : draw
    * 1 : attaccker win

### Element of Surprise

### Element of Surprise reference

-3 ~ -1 : surprise achieved by defender(-3: most, -2: substantial , -1: minor)
    
0 : Neither Side surprised
    
1 ~ 3 : surprise achieved by attacker(3: most, 2: substantial , 1: minor)

In [ ]:
plt.figure(figsize = (12, 12))
sns.set_style('whitegrid')
sns.countplot(x='wina',hue='surpa',data= surprise ,palette='RdBu_r')

### Terrain

### Terrain reference

terra 1:
    
    0 - None

    R - Rolling
    
    G - Rugged
    
    F - Flat
    
terra 2:
    
    0 - None
    
    B - Bare
    
    M - Mixed
    
    D - Desert
    
    W - Heavily Wooded
    
terra 3 :
    
    0 - None
    
    M - Marsh or Swamp
    
    U - Urban
    
    D - Dunes

In [ ]:
plt.figure(figsize = (12, 12))
sns.set_style('whitegrid')
sns.countplot(x = 'wina', hue='terra1',data= terrain ,palette='RdBu_r')

In [ ]:
plt.figure(figsize = (12, 12))
sns.set_style('whitegrid')
sns.countplot(x = 'wina', hue='terra2',data= terrain ,palette='RdBu_r')

In [ ]:
plt.figure(figsize = (12, 12))
sns.set_style('whitegrid')
sns.countplot(x = 'wina', hue='terra3',data= terrain ,palette='RdBu_r')

### Weather

### Weather Reference

wx1:
    
    D - Dry
    W - Wet
    
wx2:
    
    H - Heavy Precipitatiion
    S - Sunny (no precipitation)
    L - Light Precipitation
    O - Overcast (no precipitation)
    
wx3:
    
    H - Hot
    C - Cold
    T - Temparate
    
wx4:

    S - Summer
    $ - Spring
    W - Winter
    F - Fall
    
wx5 : 
    
    E - Tropical (i.e., "Equatorial")
    D - Desert
    T - Temperate

In [ ]:
 for j in range(1, 6):
        
    plt.figure(figsize = (12, 12))
    sns.set_style('whitegrid')
    sns.countplot(x='wina', hue='wx{}'.format(j),data= weather ,palette='RdBu_r')


### Fortification

In [ ]:
plt.figure(figsize = (12, 12))
sns.set_style('whitegrid')
sns.countplot(x='wina', hue='post1',data= fortification ,palette='RdBu_r')

In [ ]:
plt.figure(figsize = (12, 12))
sns.set_style('whitegrid')
sns.countplot(x='wina', hue='post2',data= fortification ,palette='RdBu_r')

Based on my assesment of the datasets, I'll make the following changes to the tables.

    * If there are too many missing values for imputation, it will be excluded.(terra3, post2)
    
    
    * We'll set the train and test datasets together, using Kfold.
     
    * We'll drop the columns with na values except for 'terra1':0, because we cannot manipulate the historical facts. 
    We fill in 'wx2' missing values with the most frequent values coz there's too much missing values there.
    

In [ ]:
df_combined = df[['surpa', 'post1', 'wx1', 'wx2', 'wx3', 'wx4', 'wx5', 'terra1', 'terra2', 'wina']]

display(df_combined)

msno.matrix(df_combined)
plt.show()

print(df_combined.isnull().sum())

In [ ]:
df_combined.columns

def asdf(*args):
    bomb = []
    for a in df_combined.columns:
        if a != 'wx2':
            bomb.append(a)
        
    return bomb

column_list = asdf(df_combined.columns)
print(column_list)

In [ ]:
df_combined = df_combined[column_list].dropna()
display(df_combined) #926 datas left
print(df_combined.isnull().sum())

## 1. Creating categories based on Defense Formation

 * DL - D - Delaying action adopted
 * WD - W - Withdrwal
 * FD - F - Fortified defense
 * HD - H - Hasty Defense
 * PD - P - Prepared Defense

In [ ]:
def check_post1():
    global df_combined
    
    df_combined['post1'] = df_combined['post1'].map(lambda ca: ca[0])
    #dummy encoding
    post1_dummies = pd.get_dummies(df_combined['post1'], prefix='post1')
    df_combined = pd.concat([df_combined, post1_dummies], axis=1)
    df_combined.drop('post1', inplace=True, axis=1)
    return df_combined

In [ ]:
df_combined = check_post1()
display(df_combined)

print(df_combined['post1_D'].sum())
print(df_combined['post1_F'].sum())
print(df_combined['post1_H'].sum())
print(df_combined['post1_P'].sum())
print(df_combined['post1_W'].sum())

def check_FD():
    global df_combined
    df_combined['post1'] = df_combined['post1'].map(lambda s: 1 if s == 'FD' else 0)
    return df_combined

df_combined = check_FD()
display(df_combined)
print(df_combined['post1'].sum())

## 2. Creating categories based on wx1

 * 'D':Dry → 0
 * W':Wet → 1

In [ ]:
def check_wx1():
    global df_combined
    df_combined['wx1'] = df_combined['wx1'].map(lambda s: 1 if s == 'W' else 0)
    return df_combined

In [ ]:
df_combined = check_wx1()
display(df_combined)
print(df_combined['wx1'].sum())

## 3. Creating categories based on wx3

Every value is turned into a column including only 0 or 1 values.

* C : Cold
* H : Hot
* T : Temparate

In [ ]:
def check_wx3():
    global df_combined
    
    df_combined['wx3'] = df_combined['wx3'].map(lambda ca: ca[0])
    #dummy encoding
    wx3_dummies = pd.get_dummies(df_combined['wx3'], prefix='wx3')
    df_combined = pd.concat([df_combined, wx3_dummies], axis=1)
    df_combined.drop('wx3', inplace=True, axis=1)
    return df_combined

In [ ]:
df_combined = check_wx3()
display(df_combined)
print(df_combined['wx3_C'].sum())
print(df_combined['wx3_H'].sum())
print(df_combined['wx3_T'].sum())

## 4. Creating categories based on wx4

Every value is turned into a column including only 0 or 1 values.

* S : Summer
* $ : Summer
* W : Winter
* F : Fall

In [ ]:
def check_wx4():
    global df_combined
    
    df_combined['wx4'] = df_combined['wx4'].map(lambda ca: ca[0])
    #dummy encoding
    wx4_dummies = pd.get_dummies(df_combined['wx4'], prefix='wx4')
    df_combined = pd.concat([df_combined, wx4_dummies], axis=1)
    df_combined.drop('wx4', inplace=True, axis=1)
    return df_combined

In [ ]:
df_combined = check_wx4()
display(df_combined)
print(df_combined['wx4_S'].sum())
print(df_combined['wx4_$'].sum())
print(df_combined['wx4_W'].sum())
print(df_combined['wx4_F'].sum())

## 5. Creating categories based on wx5

Every value is turned into a column including only 0 or 1 values.

* E : Tropical (i.e., "Equatorial")
* D : Desert
* T : Temperate

In [ ]:
def check_wx5():
    global df_combined
    
    df_combined['wx5'] = df_combined['wx5'].map(lambda ca: ca[0])
    #dummy encoding
    wx5_dummies = pd.get_dummies(df_combined['wx5'], prefix='wx5')
    df_combined = pd.concat([df_combined, wx5_dummies], axis=1)
    df_combined.drop('wx5', inplace=True, axis=1)
    return df_combined

In [ ]:
df_combined = check_wx5()
display(df_combined)
print(df_combined['wx5_E'].sum())
print(df_combined['wx5_D'].sum())
print(df_combined['wx5_T'].sum())

## 6. Creating categories based on terrain1

* R-Rolling
* G-Rugged
* F-Flat

In [ ]:
#creating function for creating categories based on the cabin of passengers
def check_terra1():
    global df_combined
    df_combined['terra1'] = df_combined['terra1'].map(lambda ca: ca[0])
    #dummy encoding
    terra1_dummies = pd.get_dummies(df_combined['terra1'], prefix='terra1')
    df_combined = pd.concat([df_combined, terra1_dummies], axis=1)
    df_combined.drop('terra1', inplace=True, axis=1)
    return df_combined

In [ ]:
df_combined = check_terra1()
display(df_combined)
print(df_combined['terra1_R'].sum())
print(df_combined['terra1_G'].sum())
print(df_combined['terra1_F'].sum())

## 7. Creating categories based on terrain2

* B - Bare
* M - Mixed
* D - Desert
* W - Heavily wooded

In [ ]:
#creating function for creating categories based on the cabin of passengers
def check_terra2():
    global df_combined


    df_combined['terra2'] = df_combined['terra2'].map(lambda ca: ca[0])
    #dummy encoding
    terra2_dummies = pd.get_dummies(df_combined['terra2'], prefix='terra2')
    df_combined = pd.concat([df_combined, terra2_dummies], axis=1)
    df_combined.drop('terra2', inplace=True, axis=1)
    return df_combined

In [1255]:
df_combined = check_terra2()
display(df_combined)
print(df_combined['terra2_B'].sum())
print(df_combined['terra2_M'].sum())
print(df_combined['terra2_D'].sum())
print(df_combined['terra2_W'].sum())

KeyError: 'terra2'

In [ ]:
#changing 'wina' value range from -1~1 to 0~2
df_combined['wina'] = df_combined['wina'].apply(lambda x: x+1)
df_combined['wina']
# 0: attacker loss, 1: draw, 2: attacker win 

# **Building and Training the Models**

In [ ]:
#Making X_set and Y_set
X_set = df_combined.drop('wina', axis = 1)
y_set = df_combined['wina']

In [ ]:
#Separating the train and test dataframes
x_train = X_set.copy().sample(frac = 0.7, random_state = 2)
x_test = X_set.copy().sample(frac = 0.3, random_state = 2)
y_train = y_set.copy().sample(frac = 0.7, random_state = 2)
y_test = y_set.copy().sample(frac = 0.3, random_state = 2)
x_test.reset_index(inplace=True, drop=True)
x_train.shape, x_test.shape

In [ ]:
y_test.value_counts()

<h2>1. Random Forest Model</h2>

In [ ]:
random_forest = RandomForestClassifier(max_depth=2, n_estimators=100, random_state=0)
model1 = random_forest.fit(x_train, y_train)

y_pred_random_forest = random_forest.predict(x_test)

In [ ]:
cross_val_score(RandomForestClassifier(max_depth = 2), X_set, y_set, scoring="accuracy", cv=5).mean()

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred_random_forest)

In [ ]:
from sklearn.metrics import classification_report

target_names = ['attk_loss', 'draw', 'attk_win']
print(classification_report(y_test, y_pred_random_forest, target_names=target_names))

<h2>2. Logistic Regression Model</h2>

In [ ]:
log_regres = LogisticRegression()
model2 = log_regres.fit(x_train, y_train)

y_pred_log_regres = log_regres.predict(x_test)

In [ ]:
cross_val_score(LogisticRegression(), X_set, y_set, scoring="accuracy", cv=5).mean()

In [ ]:
confusion_matrix(y_test, y_pred_log_regres)

In [ ]:
target_names = ['attk_loss', 'draw', 'attk_win']
print(classification_report(y_test, y_pred_log_regres, target_names=target_names))

<h2>3. K-nearest Neighbors</h2>

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
model3 = knn.fit(x_train, y_train)

y_pred_knn = knn.predict(x_test)

In [ ]:
cross_val_score(KNeighborsClassifier(n_neighbors=3), X_set, y_set, scoring="accuracy", cv=5).mean()

In [ ]:
confusion_matrix(y_test, y_pred_knn)

In [ ]:
target_names = ['attk_loss', 'draw', 'attk_win']
print(classification_report(y_test, y_pred_knn, target_names=target_names))

<h2>4. Gaussian Naive Bayes Model</h2>

In [ ]:
gaussian = GaussianNB()
model4 = gaussian.fit(x_train, y_train)

y_pred_gaussian = gaussian.predict(x_test)

In [ ]:
cross_val_score(GaussianNB(), X_set, y_set, scoring="accuracy", cv=5).mean()

In [ ]:
confusion_matrix(y_test, y_pred_gaussian)

In [ ]:
target_names = ['attk_loss', 'draw', 'attk_win']
print(classification_report(y_test, y_pred_gaussian, target_names=target_names))

<h2>5. Perceptrons Model</h2>

In [ ]:
perceptron = Perceptron()
model5 = perceptron.fit(x_train, y_train)

y_pred_perceptron = perceptron.predict(x_test)

In [ ]:
cross_val_score(Perceptron(), X_set, y_set, scoring="accuracy", cv=5).mean()

In [ ]:
confusion_matrix( y_test, y_pred_perceptron)

In [ ]:
target_names = ['attk_loss', 'draw', 'attk_win']
print(classification_report(y_test, y_pred_perceptron, target_names=target_names))

<h2>6. Support Vector Machine Model</h2>

In [ ]:
svc = LinearSVC()
model6 = svc.fit(x_train, y_train)

y_pred_svc = svc.predict(x_test)

In [ ]:
cross_val_score(LinearSVC(), X_set, y_set, scoring="accuracy", cv=5).mean()

In [ ]:
confusion_matrix(y_test, y_pred_svc)

In [ ]:
target_names = ['attk_loss', 'draw', 'attk_win']
print(classification_report(y_test, y_pred_svc, target_names=target_names))

<h2>7. Decision Tree Model</h2>

In [1256]:
tree = DecisionTreeClassifier()
model7 = tree.fit(x_train, y_train)

y_pred_tree = tree.predict(x_test)

In [1257]:
cross_val_score(DecisionTreeClassifier(), X_set, y_set, scoring="accuracy", cv=5).mean()

0.4827375762859634

In [1258]:
confusion_matrix(y_test, y_pred_tree)

array([[ 53,   0,  37],
       [  5,   1,  12],
       [ 30,   0, 140]])

In [1259]:
target_names = ['attk_loss', 'draw', 'attk_win']
print(classification_report(y_test, y_pred_tree, target_names=target_names))

              precision    recall  f1-score   support

   attk_loss       0.60      0.59      0.60        90
        draw       1.00      0.06      0.11        18
    attk_win       0.74      0.82      0.78       170

    accuracy                           0.70       278
   macro avg       0.78      0.49      0.49       278
weighted avg       0.71      0.70      0.68       278



<h2>8. XGBoost Model</h2>

In [1260]:
#Transforming data into Dmatrix form
d_x_train = xgb.DMatrix(x_train, label=y_train)

In [1261]:
#Setting the parameters
param = {
    'eta': 0.5, 
    'max_depth': 20,  
    'objective': 'multi:softprob',  
    'num_class': 3} 

steps = 20  # The number of training iterations

In [1262]:
#Training the model
xgb_model = xgb.train(param, d_x_train, steps)
 
y_pred_xgb = xgb_model.predict(d_x_train)
#Converting the prediction intp numpy array
y_pred_xgb_new = np.asarray([np.argmax(line) for line in y_pred_xgb])

In [1263]:
xgb_model_accuracy = round(accuracy_score(y_train, y_pred_xgb_new)*100, 2)
xgb_model_accuracy

69.91

In [1264]:
rf = RandomForestClassifier()
scores = cross_val_score(rf, x_train, y_train, cv=11, scoring='accuracy')

print("Scores:", scores)
print("Mean:", scores.mean())
print("Standard Deviation:", scores.std())

Scores: [0.59322034 0.6440678  0.62711864 0.69491525 0.6779661  0.66101695
 0.55932203 0.72881356 0.62711864 0.71186441 0.65517241]
Mean: 0.6527814675096967
Standard Deviation: 0.048081386589541614


This score shows that our model has an average accuracy of 59% with a standard deviation of around 5.1%

When using VotingClassifier, droping models with no predict_proba.
Also dropping the models with less than 0.4 cross validation scores and accuracy(on classificiation report).

In [1265]:
from sklearn.ensemble import VotingClassifier

xmodel1 = RandomForestClassifier(max_depth=1, n_estimators=100, random_state=0)
xmodel2 = LogisticRegression(random_state=1)
xmodel3 = KNeighborsClassifier(n_neighbors=3)
#xmodel4 = GaussianNB()
#xmodel5 = Perceptron()
#xmodel6 = LinearSVC()
xmodel7 = DecisionTreeClassifier()

ensemble = VotingClassifier(estimators=[('rf', xmodel1), ('lr', xmodel2), ('knn', xmodel3),
                                       ('nb', xmodel4), ('dt', xmodel7)], voting='soft')

ensemble2 = ensemble.fit(x_train, y_train)
y_pred_ensemble = ensemble2.predict(x_test)

In [1266]:
cross_val_score(ensemble, X_set, y_set, scoring="accuracy", cv=5).mean()

0.5616216216216217

In [1267]:
confusion_matrix(y_test, y_pred_ensemble)

array([[ 41,   0,  49],
       [  3,   1,  14],
       [ 19,   0, 151]])

In [1268]:
target_names = ['attk_loss', 'draw', 'attk_win']
print(classification_report(y_test, y_pred_ensemble, target_names=target_names))

              precision    recall  f1-score   support

   attk_loss       0.65      0.46      0.54        90
        draw       1.00      0.06      0.11        18
    attk_win       0.71      0.89      0.79       170

    accuracy                           0.69       278
   macro avg       0.79      0.47      0.48       278
weighted avg       0.71      0.69      0.66       278



<h2>Feature Importance</h2>

In [1269]:
importances = pd.DataFrame({'Feature':x_train.columns,'Importance':np.round(random_forest.feature_importances_,3)})
importances_df = importances.sort_values('Importance', ascending=False).reset_index(drop=True)
importances_plot = importances.sort_values('Importance', ascending=False).set_index('Feature')
importances_plot.head()

,Importance
Feature,
surpa,0.259
terra2_B,0.118
post1_H,0.109
wx4_$,0.104
wx4_F,0.103


In [1270]:
#Plotting the Features importances
fig_bar_feature = go.Figure()
fig_bar_feature.add_trace(go.Bar(x=importances_df['Feature'], y=importances_df['Importance']))
fig_bar_feature.update_layout(width=800, title_text='Importances of Features',
                       xaxis_title_text='Features', yaxis_title_text='Importance', xaxis_tickangle=-45)
fig_bar_feature.show()

From the graph we can see that all the categories after "wx3_T" and onwards doesn't play an important role in the prediction, hence dropping those categories.

In [1271]:
for i in range(10, importances_df.shape[0]):
    column = importances_df['Feature'][i]
    X_set.drop([column], inplace=True, axis=1)

display(X_set)

,surpa,post1_D,post1_F,post1_H,wx3_C,wx3_H,wx3_T,wx4_$,wx4_F,terra2_B,terra2_D
isqno,,,,,,,,,,,
1,0.0,0,0,1,0,0,1,0,0,1,0
2,0.0,0,0,1,0,0,1,0,0,0,0
3,0.0,0,0,1,0,1,0,1,0,0,0
5,0.0,0,0,1,0,0,1,0,0,0,0
6,0.0,0,0,1,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
654,0.0,0,0,0,0,0,1,1,0,0,0
655,0.0,0,0,0,0,0,1,1,0,0,0
656,0.0,0,0,0,0,1,0,0,0,1,0


In [1272]:
#Separate the train and test dataframes
#Separating the train and test dataframes
x_train = X_set.copy().sample(frac = 0.7, random_state = 2)
x_test = X_set.copy().sample(frac = 0.3, random_state = 2)
x_test.reset_index(inplace=True, drop=True)

In [1273]:
random_forest = RandomForestClassifier()
random_forest.fit(x_train, y_train)
y_prediction = random_forest.predict(x_test)


accuracy_random_forest = round(random_forest.score(x_train, y_train)*100, 2)
print("The accuracy after removing least important Features is {}, which is same as before removing.".format(accuracy_random_forest))

The accuracy after removing least important Features is 70.22, which is same as before removing.


<h2>Training Decision Tree Model Again</h2>

In [1274]:
tree = DecisionTreeClassifier()
tree.fit(x_train, y_train)

y_pred_tree = tree.predict(x_test)

tree_accuracy = round(tree.score(x_train, y_train)*100, 2)
print("The accuracy after removing least important Features is {}, which is same as before removing.".format(tree_accuracy))

The accuracy after removing least important Features is 70.22, which is same as before removing.


<h2>Training XGBoost Model Again</h2>

In [1275]:
#Transforming data into Dmatrix form
d_x_train = xgb.DMatrix(x_train, label=y_train)
d_x_test = xgb.DMatrix(x_test)

In [1276]:
xgb_model = xgb.train(param, d_x_train, steps)
y_pred_xgb = xgb_model.predict(d_x_test)
y_pred_xgb_new = np.asarray([np.argmax(line) for line in y_pred_xgb])


# Submission

> # Summary

This project enhanced my Machine Learnin